KoGPT2 파인튜닝 https://github.com/SKT-AI/KoGPT2

### 구글 마운트

In [1]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

# 구글 드라이브 경로 설정
%cd /content/drive/MyDrive/텍스트 프로젝트
%pwd

Mounted at /content/drive
/content/drive/MyDrive/텍스트 프로젝트


'/content/drive/MyDrive/텍스트 프로젝트'

### 라이브러리 설치

In [2]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.6 MB/s eta 0:00:00


### 라이브러리 불러오기

In [3]:
import pandas as pd
# for padding
# from tensorflow.keras.preprocessing.sequence import pad_sequences

import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW
from tqdm import tqdm

import time
import datetime

# 토크나이저
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
                bos_token='</s>', eos_token='</s>', unk_token='<unk>',
                pad_token='<pad>', mask_token='<mask>')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


### 모델 테스트


In [ ]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
bos_token='</s>', eos_token='</s>', unk_token='<unk>',
pad_token='<pad>', mask_token='<mask>')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [ ]:
tokenizer.tokenize("안녕하세요. 한국어 GPT-2 입니다.😤:)l^o")

['▁안녕',
 '하',
 '세',
 '요.',
 '▁한국어',
 '▁G',
 'P',
 'T',
 '-2',
 '▁입',
 '니다.',
 '😤',
 ':)',
 'l^o']

In [ ]:
import torch
from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

In [ ]:
text = '근육이 커지기 위해서는'
input_ids = tokenizer.encode(text, return_tensors='pt')
gen_ids = model.generate(input_ids,
                        max_length=128,
                        repetition_penalty=2.0,
                        pad_token_id=tokenizer.pad_token_id,
                        eos_token_id=tokenizer.eos_token_id,
                        bos_token_id=tokenizer.bos_token_id,
                        use_cache=True)
generated = tokenizer.decode(gen_ids[0])
print(generated)

근육이 커지기 위해서는 무엇보다 규칙적인 생활습관이 중요하다.
특히, 아침식사는 단백질과 비타민이 풍부한 과일과 채소를 많이 섭취하는 것이 좋다.
또한 하루 30분 이상 충분한 수면을 취하는 것도 도움이 된다.
아침 식사를 거르지 않고 규칙적으로 운동을 하면 혈액순환에 도움을 줄 뿐만 아니라 신진대사를 촉진해 체내 노폐물을 배출하고 혈압을 낮춰준다.
운동은 하루에 10분 정도만 하는 게 좋으며 운동 후에는 반드시 스트레칭을 통해 근육량을 늘리고 유연성을 높여야 한다.
운동 후 바로 잠자리에 드는 것은 피해야 하며 특히 아침에 일어나면 몸이 피곤해지기 때문에 무리하게 움직이면 오히려 역효과가 날 수도 있다.
운동을


### 사용자 정의 함수

In [4]:

import re

# 가사 전처리
def preprocess(sentences):
    lst = []
    for sentence in sentences:
        # \n과 \t를 제거한다
        sentence = re.sub('\t', '', sentence)
        sentence = re.sub('\n', '', sentence)
        # 한국어, 공백 빼고 제거하기
        # sentence = re.sub('[^ㄱ-ㅎ가-힣\s]+', '', sentence)
        # 문장 양옆의 띄어쓰기를 지운다
        sentence = sentence.strip()

        lst.append(sentence)
    return lst

### 데이터 불러오기

In [5]:
song_df = pd.read_excel('가사_all.xlsx')
song_df['Lyrics'] = preprocess(song_df['Lyrics'])

In [6]:
song_df['Lyrics']

0       어린 햇살 아래서뛰어놀곤 했었던가쁜 숨결굽이진 골목 지나길을 따라가보면같은 기억어른...
1       꽃이 언제 피는지 그딴 게 뭐가 중요한데날씨가 언제 풀리는지 그딴 거 알면 뭐 할 ...
2       늦게 다니지좀 마술은 멀리좀 해봐열살짜리 애처럼 말을 안듣니정말 웃음만 나와누가 누...
3       난 또 행복한 척 더 더 행복한 척난 또 행복한 척 더 더 행복한 척아무에게도 말하...
4       밥을 먹어도 혼자야구를 봐도 혼자거리를 걸어도 나 혼자혼자이기 싫은데또 방에 누워도...
                              ...                        
3341    잠에서 막 일어난 얼굴도 귀여워잠겨버린 목소리 그것도 귀여워너는 아마 그것까진 모를...
3342    아무말도 없이 날 바라보는게뭔가 이상해서 두귀를 막았어내 눈에 보이는 너의 입모양이...
3343    This is the tihs that y'all ain't ready for Th...
3344    하루 종일 비가 와서니 얼굴이 또 생각 나 눈물이 났어한 걸음 더 걸어가면 잊혀질까...
3345    관둬 가차없이 너랑 끝낼래너같은 남잔 필요없어대체 그럴꺼면 왜 나를 만나니딴 여자랑...
Name: Lyrics, Length: 3346, dtype: object

### DataLoader

In [7]:
lst = ['사랑은 눈에 보이지가 않아어디에 있는지 몰라더듬거리다 찾아다니다이제야 손 끝에 닿았나 봐그때 그대 뒤를 돌아서내게 짓는 웃음에 난 눈을 떠 바람에 흩날리는 그대 향기가낮게 들려오는 그대 목소리가겨울처럼 차가운 내 맘그 안에 스며 들어 와사랑이 눈꽃으로 피어 나그렁 그렁 눈물이 고여이 사랑이 언젠가 시들까 봐바람에 흩날리는 그대 향기가낮게 들려오는 그대 목소리가겨울처럼 차가운 내 맘그 안에 스며 들어 와사랑으로 피어 나하루 이틀 사흘 나흘내 시간을 다 쓸 만큼그대 하나로 난 행복 해바람에 흩날리는 그대 향기가낮게 들려오는 그대 목소리가겨울처럼 차가운 내 맘그 안에 스며 들어 와사랑이 눈꽃으로 피어 나']

In [8]:
len(lst[0]) # 338

338

In [9]:
song_df['Lyrics']

0       어린 햇살 아래서뛰어놀곤 했었던가쁜 숨결굽이진 골목 지나길을 따라가보면같은 기억어른...
1       꽃이 언제 피는지 그딴 게 뭐가 중요한데날씨가 언제 풀리는지 그딴 거 알면 뭐 할 ...
2       늦게 다니지좀 마술은 멀리좀 해봐열살짜리 애처럼 말을 안듣니정말 웃음만 나와누가 누...
3       난 또 행복한 척 더 더 행복한 척난 또 행복한 척 더 더 행복한 척아무에게도 말하...
4       밥을 먹어도 혼자야구를 봐도 혼자거리를 걸어도 나 혼자혼자이기 싫은데또 방에 누워도...
                              ...                        
3341    잠에서 막 일어난 얼굴도 귀여워잠겨버린 목소리 그것도 귀여워너는 아마 그것까진 모를...
3342    아무말도 없이 날 바라보는게뭔가 이상해서 두귀를 막았어내 눈에 보이는 너의 입모양이...
3343    This is the tihs that y'all ain't ready for Th...
3344    하루 종일 비가 와서니 얼굴이 또 생각 나 눈물이 났어한 걸음 더 걸어가면 잊혀질까...
3345    관둬 가차없이 너랑 끝낼래너같은 남잔 필요없어대체 그럴꺼면 왜 나를 만나니딴 여자랑...
Name: Lyrics, Length: 3346, dtype: object

In [10]:
class SongDataset(Dataset):
    def __init__(self, tokenizer, data):
        lst = [str(i) for i in data.values]

        inputs = tokenizer(
            lst,
            return_tensors="pt",
            padding="max_length",
            truncation=True,
            max_length=300,
        )

        self.item = inputs["input_ids"]
        self.attention_mask = inputs["attention_mask"]

        self.length = len(lst)

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        return self.item[idx], self.attention_mask[idx]

In [11]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
                bos_token='</s>', eos_token='</s>', unk_token='<unk>',
                pad_token='<pad>', mask_token='<mask>')

print('Lyrics의 최대 길이: ', max([len(lyrics) for lyrics in song_df['Lyrics']]))

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


Lyrics의 최대 길이:  2516


In [12]:
tokenizer.tokenize('가나다')

['▁가', '나다']

In [13]:
batch_size=2

train_data = SongDataset(tokenizer, song_df['Lyrics'])
train_dataloader = DataLoader(train_data, batch_size=batch_size)

In [14]:
# 데이터셋 잘 만들어졌는지 확인
train_data.__getitem__(3)

(tensor([ 9769,  9108, 29918, 11220,  9267,  9267, 29918, 11220,  7058,  9108,
         29918, 11220,  9267,  9267, 29918, 11220,  7965,  7556, 19387, 50521,
         11335, 13103,  8146,  9341,  8146, 10431, 14042,  6889, 12158, 21871,
         30514,  9366, 11525,  8719,  7653,  8711, 11295, 24692,  9049,  6872,
          9176,  7089, 11941,  9689, 11389, 20937,  6958,  7198,  9038, 20849,
         17582,  8146, 10431, 10341,  9566,  8135, 47318,  8006,  7060, 38142,
          9080, 42399,  9136, 18256, 21871, 31187,  6958,  9033,  9135,  9136,
         29500,  8102,  7058,  9108, 29918, 11220,  9267,  9267, 29918, 11220,
          9033, 17582, 16238, 13761, 10056, 13358,  8137,  9068, 11523,  6824,
          9222,  8095,  9068,  6958, 41508, 13358,  8137,  9068, 47588,  6889,
         13358,  8137,  9068,  7058,  9108, 29918, 11220,  9267,  9267, 29918,
         11220,  9033, 17582, 39417, 39502,  9154, 15651, 12371,  9923,  8420,
          8015,  9176,  8146, 12114, 17272, 10099,  

In [15]:
# 데이터로드 잘 만들어졌는지 확인
dataiter = iter(train_dataloader)
item, attention_mask = next(dataiter)

In [16]:
item

tensor([[10452, 15749,  7760, 43468,  7348,  8006,  7122,  6891,  9651, 12016,
          6824,  7749, 12154,  6879,  6927,  8146,  8265, 10187,  7541, 10051,
         16535, 37909, 18381, 11891, 14337,  8006,  7469, 10479, 32392,  7965,
          8264,  9105,  7770, 25715, 21871, 20668, 11508, 10137,  9202,  8718,
          7182,  9098,  8030,  6960, 10554,  8148,  6857,   431, 15949,   681,
         22985, 13063,   443, 10630, 11370, 11780, 43618,  7307,  8022, 12761,
          8006, 10624, 12306, 30903, 10099,  8066,  9769,  9018,  8198, 26029,
          6828,  9470, 19152,  7061,  7965,  8265, 48047, 10809,  8160, 12306,
          9071,  7429, 14337,  8711,   429,   446,  9714, 10054,   681,   451,
         13063,  9837, 42062, 18125, 36955, 48885,   461, 14778,   457, 21843,
          7372,  9769, 15076,  7599,  8277, 16005,  9873,  9174,  9847,  7487,
          8236,  9049,  7210, 35860,  7487,  8236,  9049,  7210, 15378,  8030,
          6958, 11780,  9768, 22985, 11780, 43618, 3

In [17]:
attention_mask

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0

### GPU 사용

In [18]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


### 모델 만들기

In [19]:
import torch
from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2').to(device)

In [20]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

In [21]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5,
                  eps = 1e-8)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [22]:
def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))  # hh:mm:ss

In [52]:
load_data = torch.load("model_kogpt2/kogpt2_fine-tunning_51.pt")

In [53]:
load_data.keys()

dict_keys(['epoch', 'loss', 'model', 'optimizer'])

In [54]:
load_epoch = load_data['epoch']
load_epoch

51

In [60]:
load_data['loss']

0.173088937997818

In [55]:
# 저장한 모델 불러오기
# 모델 로드
model.load_state_dict(load_data['model'])
# 옵티마이저 로드
optimizer.load_state_dict(load_data['optimizer'])

In [56]:
# cuda 메모리 지우기
import torch
torch.cuda.empty_cache()

import gc
gc.collect()

45

In [57]:
def generate(keyword):
    input_ids = tokenizer.encode(keyword, return_tensors='pt')
    input_ids = input_ids.to(device)
    gen_ids = model.generate(input_ids,
                        max_length=64,
                        repetition_penalty=2.0,
                        pad_token_id=tokenizer.pad_token_id,
                        eos_token_id=tokenizer.eos_token_id,
                        bos_token_id=tokenizer.bos_token_id,
                        use_cache=True)
    generated = tokenizer.decode(gen_ids[0])
    return generated

In [41]:
# 학습 전 generate 함수 동작 테스트
keywords = ['밥을 먹어도', '매일같이', '배고파']

for keyword in keywords:
    #keyword = keyword.to(device)
    gen = generate(keyword)
    gen_print = gen.split('\n')[0:2]
    print(f"{keyword} : {gen_print}")

밥을 먹어도 : ['밥을 먹어도 혼자야굴에 엎드려 힘겨워도너 하나면 되는거니슬픔이 와도가끔 내게 말해도조금만 잘 지내줘별일 없이 하루를 보내자난 널 믿어 버릴 수 있을테니까내 품에서 잠들고 싶어정말 괜찮아 행복했었']
매일같이 : ['매일같이 영화 속에서나 책 속에난 그저 드라마 속떠도는 여자 마치 오백 년 달콤해 너와 함께한 시절이 아직도 내겐 그렇듯그때는 상상하지 않았고 사실 난 널 좋아했어우리 처음 만난 날 이후로두 사람은 서로 화제를 주고받지 못했지몰랐지만 지금은 생각해보면']
배고파 : ['배고파 식어 먹자 안돼안되어난 고작 단 한입만 먹을 수 있어그런데 왜 나는 네가 좋아친구야 뭐해정말 좋다 생각했었는걸 그래 전부 소중하게드릴게 그것뿐이야뭐든지 중요하지도 않아니가 원하는 게 무엇이든 간']


In [58]:
load_epoch

51

In [61]:
# 학습 모드
model.train()
min_loss = 1
epoch_num = 5

min_loss = load_data['loss']

for epoch in range(load_epoch + 1, epoch_num+load_epoch):
    print('======== Epoch {:} / {:} ========'.format(epoch, epoch_num+load_epoch))
    t0 = time.time()
    total_loss = 0

    for input_text, attention_mask in tqdm(train_dataloader):
        input_tensor = input_text.to(device)
        attention_mask = attention_mask.to(device)
        # print(input_tensor)
        # print(input_tensor.shape)

        # input과 label이 같다.
        outputs = model(input_tensor, labels=input_tensor, attention_mask=attention_mask)
        # print('outputs:', outputs)
        loss = outputs[0]

        # cpu로 변경하기
        input_tensor.cpu()

        optimizer.zero_grad()
        model.zero_grad()
        loss.backward()
        optimizer.step()

    now_loss = outputs[0].item()
    print(f"epoch {epoch} loss {now_loss:0.2f}")
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))

    # loss가 min_loss보다 작을 경우
    if now_loss < min_loss:
        min_loss = now_loss
        # 모델 저장
        torch.save({'epoch':epoch,
                    'loss':min_loss,
                    'model':model.state_dict(),
                    'optimizer':optimizer.state_dict()},
                   f"model_kogpt2/kogpt2_fine-tunning_{epoch}.pt")

    # 문장 출력해보기
    keywords = ['밥을 먹어도', '매일같이', '배고파', '학원에서']
    for keyword in keywords:
        gen = generate(keyword)
        gen_print = gen.split('\n')[0:2]
        print(f"{keyword} : {gen_print}")


======== Epoch 52 / 56 ========


100%|██████████| 1673/1673 [05:57<00:00,  4.68it/s]


epoch 52 loss 0.17
  Training epcoh took: 0:05:58
밥을 먹어도 : ['밥을 먹어도 혼자야구를 봐도 나 혼자서혼자서 먹을 뿐인데또 방에 누워있다가혹시 니가 또 만나면그 땐 내가 먼지떨려울지도 몰라아무 표정 없는 얼굴로난 너를 보며 다시 한 번웃어줘이제 그만 좀 해봐우리 이제']
매일같이 : ['매일같이 영화 속에서나 책속에서나의 모습을 느껴 UhBeep burning my love날 안아 봐도 너무나 멋진그런 너를앞으로 열하게 됐어너무 멋진 말투지만자기야 내 이런 얼굴까지떠난 줄 알았어사랑이 정말 좋았을까내']
배고파 : ["배고파서밥 한 끼 해주고 싶은데난 아직 여전해어이없어서 now우린 아무것도 필요 없어인생은 불공평해서웃저버린 삶이야술 마시면 자꾸 내가 생각나괜히 조급해져사실 난 이게 좋아지금 타고 있는 길이i'm sti"]
학원에서 : ['학원에서 일하고몇 달을 벌어도 몇 십 년 애인 걸네 하지만 내 생에 넌 없다는 거 그 사실을 감사할 일 없지만 볼 수 없었던 나 나의 모습을사랑하고 싶지만 너무도 부족한나의 삶에 한줄기 빛처럼 박힌그 시절꽃잎들처럼 여전히 꽃들의 꽃잎 처럼보여지고 지워지지']
======== Epoch 53 / 56 ========


100%|██████████| 1673/1673 [06:10<00:00,  4.52it/s]


epoch 53 loss 0.13
  Training epcoh took: 0:06:10
밥을 먹어도 : ['밥을 먹어도 혼자야구를 봐도혼자만 있으면 괜찮아요즘엔 왜 자꾸 니 생각이 나떠난 줄 알면서도어떻게 널 보냈는지이렇게 다들 그렇게내 맘은 편안해졌나봐답장을 해줘 어젯밤네 생각에 걸렸어나를 사랑한다']
매일같이 : ["매일같이 영화 속에서 사랑을 느껴 Um 이렇게도 가슴이 뛰는 건 처음이니까 나혼자 걷다 자꾸 멈추곤 해 아직 네가 없는 하루 빈자리만 잘 살아서 미안해 내 탓일까 봐 너무 보고 싶어서 You're my love 넌 나를 떠나지마 지금"]
배고파 : ['배고파서 노래라도 하다 벌써 새벽인데 잠도 안 오고전화기만 놔두면 있잖아 내가할 말이 있어그냥 앉아있어줄게 없단 걸 알아난널 놓을 수 없어 그 자리에 널 남겨둔 채사랑하고 싶지만 마음뿐인걸 미안해 이 말 밖에']
학원에서 : ['학원에서 모두 퇴근길 쯤에 두고 건네오는 커피를 참 오래머물러다 주말에 놀던 모닝콜을 외면하고선미리 서둘러 내린 셔츠 안을 여자들은늘 똑같은 걸따라해마다 겨울이 와왠지 살살 녹아 내리는슬픈 내 맘에그 겨울의 끝을 난']
======== Epoch 54 / 56 ========


100%|██████████| 1673/1673 [06:06<00:00,  4.56it/s]


epoch 54 loss 0.13
  Training epcoh took: 0:06:07
밥을 먹어도 : ['밥을 먹어도 혼자야구를 봐도 혼자서거리를 걸어가눈치밥먹지도 못하고다가정말 정말로 외로운건메뉴판 보며 나 그리워진게좀더 예뻐져야만 내가 살아볼께원래 이 노래는내가 지기전에 잊어버려친구들 다 불러내봐아']
매일같이 : ['매일같이 영화 속에서나 책 속에나는 떠오르던 네가지하루에 몇 번씩이나 하늘을 나서는 법도 몰라아무도 아무것도 모른 채 그저 멍하니 서있는 날하늘은 내 생각만 하고매일 잠들지 못하고바람이 불면마자 일어나창문을 열면 기대어너의 햇살과 커피 냄새']
배고파 : ['배고파서밥을 먹든 커피한잔 하자니까 나랑 있는 그대로여기 내 앞에서 당신과나 같이 앉아있네다른 행선지도 걸렸어우연히라도 마주칠까난 먼저 말을 건넸지괜히만 두근거려서그럼 난 어떡할래아무말도']
학원에서 : ['학원에서 늘 하룰은교실에 앉아있을 땐햇살 받으며하지만 내 마음은 항상 바쁘고그런 너를 위해 장미꽃을 들고말없이 기다리다노오란 은행잎들을 흔들면서모두가 나를 대망하는 지금내 마음 흔들어 놓고간 그대 너무나 사랑했던 만큼힘이 들때']
======== Epoch 55 / 56 ========


100%|██████████| 1673/1673 [06:10<00:00,  4.52it/s]


epoch 55 loss 0.14
  Training epcoh took: 0:06:10
밥을 먹어도 : ['밥을 먹어도 혼자야구를 봐도 혼자서거리를 걸어가도나에게서 자유로와미소로 눈을 뜨고주먹을 쥐고서늦은 저녁 일이와할 말이 있어도아직 많이 기다렸어혼자 두지 않기로 했잖아이 거리에서나에게 무슨 말이라도 해주오이럴 땐 뭐가 있어소']
매일같이 : ['매일같이 영화 속에서나 책상에 앉아음악을 봐들을 때코끝을 맴도는 나의 텅빈 머릿속에나의 모습 그대라는 꿈을 꾸게도그대는 내 옆에 아무도 없는떠나간 많은 사람들언제부턴가 내게 다가와인기하게 됐어요새로움을 잃어 버린 것뿐인데너와 함께']
배고파 : ['배고파 소리 높여 앉아야배우고 싶어 식사는 하고 싶은 거참을 수다 떨 시간 내 뒤에인생이란 기분이틀에 젖어 우는데나의 투박한 마음 가득히돈 쓸 필요 없어그저 내게 머물러만 줘다시 한번 걸어갈게웃음 가득한 너의 잔소리사랑']
학원에서 : ['학원에서 택시에서 댈 기다리다가 마주친 눈빛 너는 내 맘 모르겠지 차창가에 앉아 오감싸며 행복을 얘기하는 작은 기도를 아무렇지도 않은 듯 간단하게 인형을 눌러 나에게 인사하고파 수줍게 건넨 인사를 하며 미안한 마음도 보이지 않고 그 무슨 일이라도 생긴']


In [49]:
# 학습 후 generate 함수 동작 테스트
keywords = ['밥을 먹어도', '매일같이', '배고파', '학원에서']

for keyword in keywords:
    #keyword = keyword.to(device)
    gen = generate(keyword)
    gen_print = gen.split('\n')[0:2]
    print(f"{keyword} : {gen_print}")

밥을 먹어도 : ['밥을 먹어도 혼자야구를 봐도 혼자서거리를 걸어가며 하루를 살아오네홀로 남겨진 내 모습하염없이 눈물이 흘러이제 다시 볼 수 없겠지만그땐 한번쯤은날 보며 웃어 주던 너수줍음에눈물을 흘리며 돌아서 여기까지 온힘겨']
매일같이 : ['매일같이 영화 속에서나 책 속에나의 모습을 느껴요하루의 빈자리까지 다 채워 주고 싶어내겐 너 아니면 안 되는 날왜 이리 바보 같은지어떤 사람 곁에 있어도나는 그게 너무 편했어내가 사랑한너란 사람이언제나 내게 가끔씩 주는 인사며칠이 지나도 계속 옆에']
배고파 : ['배고파도 보고싶어 갑갑산 산마루에 앉으면설레이는 마음그런 모든 사람들놓고 위로가자꾸 맘이 답답한행복한 사람아입상에 누워서 울적하는 한숨나는사람은 없으니까배 고픈 밤인데너는 어디로 가서뭐 그리 멀쩡하게']
학원에서 : ['학원에서 유치원 과정까지 열아홉솔직히 말하자면원래 힙합을 나는 아이였고그게 나의 희망이었다네이게 우리의 첫걸이었던 그땐뭐 몰랐었지지금은 상상해만 싶은데 난 정말이지난 일로 엮여버린 놈들이 전부였지괜찮은 서로의 생각들만 가득했던']


In [50]:
keywords = ['사랑', '그대', '생각', '사람']

for keyword in keywords:
    #keyword = keyword.to(device)
    gen = generate(keyword)
    gen_print = gen.split('\n')[0:2]
    print(f"{keyword} : {gen_print}")

사랑 : ['사랑해 내 인생이 다할때까지고마워 나를 믿고 달려온거란걸 알수없어 나없이 뛰어너를 쓰러지게 한건니가 아니야니 인생에 전부 바넣은 사람난 너뿐이야그런 니가 원망스러워정말 보고싶어서사랑하는 널 보면서내게']
그대 : ['그대만 보면 난 딴 세상에나 혼자 있는 것 같아요그대 내 맘을 말해줄래요일마다 나 생각해해맑은 미소로 나를 반겨 줄게날 바라보는 네 눈빛 수줍던 입술까지도우리 함께 했던 그 날 모든 게 그대로인걸너무 좋아서 그랬어시간이란 흘러가고']
생각 : ['생각나니 처음 우리너무 멋쩍 고개 끄덕였지내가 아는 모든 것들나를 원한다 하면꺼내본 적 없는 얘기누군가 들려오는 어제했던 일처럼말하고 싶지만 아님 어디서부터 어떻게 시작해야할까천년 사는 내 사람아그렇게 태어나 너를 사랑한']
사람 : ['사람들아 함께 어느새 잊었나봐요그대의 추억만 남아있죠내게 미련이란 걸맘을 달래려고 그댈 만났지만떠날 수 밖에 없는 나인데언제까지 가슴시린지 오래겠네요한번쯤은 이별 아닌 척해 본심을 숨기지 못해아파도 좋아']


In [51]:
keywords = ['사랑', '그대', '생각', '사람']

for keyword in keywords:
    #keyword = keyword.to(device)
    gen = generate(keyword)
    gen_print = gen.split('\n')[0:2]
    print(f"{keyword} : {gen_print}")

사랑 : ['사랑해줘아무런 조건없이어디라도 좋아시작이 몇년이야나는 너무 바쁜 몸만 가졌지몇년이 지났는지 몰라그 어느새 Goodbye넌 골골대고 너는 창큰 사람내겐 처음선물이었네 내여자가 되는 날까지Got']
그대 : ['그대 사랑했죠아무일도 아닌 일에 웃고있는 나를 보면서 깨달은 것은 아무것도 없어요이젠 그댈 놓아주세요가끔 한 번씩은그댄 추억에 잠들어하루가 지나고 또 다시 찾아와마치 아무 일도 없는 듯나와 마주치는어색한 미소만애써 웃음 지어 보여그러나 이']
생각 : ['생각도 못했던 말내게 니 모습이 보이고 사랑했던 순간들헤어져 있던 만남이젠 꿈만 같아너무 쉽게 끝난걸 이제야조금 더 기다려봐내가 나를 알겠니잘 지내왔는지 나보다어떻고 너를 그리워했었지함께한 시간이나 많이 늦었는데참 행복했는데']
사람 : ['사람들아 기죽지 마라당신없는 이 사랑은한동안은 우린 편하겠지만그렇게 하지마우리는 왜 이렇게도간당할 수 없는 사랑을가슴바라만 봐도가슴 아픈 사랑으로왜 나를 아프게 하나스쳐 지나가는 바람처럼우리 헤어짐이란 두 글자가이젠 나의 가슴에 쓰러져']
